In [22]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt



# CRUD Python module file name and class name
from AnimalShelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# Username and password and CRUD Python module name

username = "root"
password = "YpRfY5f0Ky"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invalid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

# Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' # Name of the file 
encoded_image = base64.b64encode(open(image_filename, 'rb').read())


app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    # Class name
    html.Center(html.B(html.H1('CS-340 Dashboard'))),
    ## HTML Grazioso Salvare's image logo
    html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))),
    
    ## Unigue Identifier containing the creator's name
    html.H1("Creator: Jasper Conneway. Date: 12/16/2024", style={'font-size':15}),
    html.Hr(),
    html.Div([ ## Checkboxes to allow filtering options
        dcc.Checklist(
        ['Water Rescue', 'Mountain Rescue', 'Disaster Rescue', 'Reset'],
        [], id="filter-type", # id for callback
        inline=True # checklist in a row
        )
    ]
    ),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        
        ## Features for interactive data table to make it user-friendly for the client
        editable=False,
        # allow filter by checklist
        filter_action="custom",
        filter_query="",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable=False,
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current=0,
        page_size=10
       ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback(Output('datatable-id','data'),
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
## Code to filter interactive data table with MongoDB queries
    # Initialize df to an empty DataFrame
    df = pd.DataFrame()
    
    if filter_type == 'Water Rescue':
        ## Breeds must be lab, chesapeake, or newfoundland
        ## Omit 'mix' in the query
        df = pd.DataFrame.from_records(db.read({
            "animal_type":"Dog","breed":{"$in":["Labrador Retriever","Chesapeake Bay Retriever","Newfoundland"]},
              "sex_upon_outcome":"Intact Female",
              "age_upon_outcome_in_weeks":{"$gte":26},"age_upon_outcome_in_weeks":{"$lte":156}}))

    elif filter_type == 'Mountain Rescue':
        ## Query the database for mountian rescue preferences
        df = pd.DataFrame.from_records(db.read({
            "animal_type":"Dog", "breed":{"$in":["German Shepherd","Alaskan Malamute","Old English Sheepdog","Siberian Husky", "Rottweiler"]},
                "sex_upon_outcome":"Intact Male", 
                "age_upon_outcome_in_weeks":{"$gte":26},"age_upon_outcome_in_weeks":{"$lte":156}}))
        
    elif filter_type == 'Disaster Rescue':
        ## Query the database for disaster rescue preferences
        df = pd.DataFrame.from_records(db.read({
            "animal_type":"Dog","breed":{"$in":["Doberman Pinscher","German Shepherd","Golden Retriever","Bloodhound","Rottweiler"]},
            "sex_upon_outcome":"Intact Male",
            "age_upon_outcome_in_weeks":{"$gte":20},"age_upon_outcome_in_weeks":{"$lte":300}}))
    
    ## No filters on data table
    elif filter_type == 'Reset':
        df = pd.DataFrame.from_records(db.read({}))
        
    df.drop(columns=['_id'],inplace=True)
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    data=df.to_dict('records')
    
    ## return the requested data table
    return (data,columns)


# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    ## Check if viewData is empty
    if viewData is None:
        return
    df = pd.DataFrame.from_dict(viewData)
    
    # add code for chart of your choice (e.g. pie chart) #
   
    return [
        dcc.Graph(            
            figure = px.pie(df, names='breed', title='Preferred Animals')
            
        )    
    ]
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    if viewData is None or not viewData:
        return html.P("No data available to display on the map.")

    dff = pd.DataFrame.from_dict(viewData)

    if dff.empty:
        return html.P("No data available to display on the map.")

    # Handle default index
    row = index[0] if index else 0

    # Check if row index is within bounds
    if row >= len(dff):
        return html.P("Selected row is out of range.")

    # Debugging prints
    print("DataFrame shape:", dff.shape)
    print("Selected row index:", row)
    print("Row data:", dff.iloc[row])

    # Ensure the DataFrame has the correct columns for latitude and longitude
    if 'location_lat' not in dff.columns or 'location_long' not in dff.columns:
        return html.P("Latitude or longitude columns are missing.")

    if pd.isnull(dff.iloc[row]['location_lat']) or pd.isnull(dff.iloc[row]['location_long']):
        return html.P("Invalid coordinates for the selected data.")

    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75, -97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[dff.iloc[row]['location_lat'], dff.iloc[row]['location_long']], children=[
                dl.Tooltip(dff.iloc[row]['breed']),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row]['name'])
                ])
            ])
        ])
    ]



app.run_server(debug=True)


Connection to MongoDB initialized successfully
Dash app running on http://127.0.0.1:14044/
DataFrame shape: (10000, 16)
Selected row index: 0
Row data: rec_num                                           2
age_upon_outcome                             1 year
animal_id                                   A725717
animal_type                                     Cat
breed                        Domestic Shorthair Mix
color                                  Silver Tabby
date_of_birth                            2015-05-02
datetime                        2016-05-06 10:49:00
monthyear                       2016-05-06T10:49:00
name                                               
outcome_subtype                                SCRP
outcome_type                               Transfer
sex_upon_outcome                      Spayed Female
location_lat                              30.652598
location_long                            -97.741996
age_upon_outcome_in_weeks                 52.921528
Name: 0, dtype: 